In [13]:
import pandas as pd
import json

df = pd.read_csv('../data/clean.csv')
meta = pd.read_csv('../data/feature_metadata.csv')

multi_features = [
    'other_activities',
    'hangboard_grips',
    'hangboard_style',
    'endurance_style',
    'strength_style'
]
for feature in multi_features:
    df[feature] = [eval(i) if i==i else None for i in df[feature].values ]

key2label = {k:v for k, v in meta[['DataFrameKey', 'Label']].values}

sorted_years = [
    '<1 year','1-2 years', '2-3 years', '3-4 years', '4-5 years', '5-6 years',
    '6-7 years', '7-8 years', '8-9 years', '9-10 years', '10-11 years',
    '11-12 years', '12-13 years', '13-14 years', '14-15 years', '15+ years'
]

strength_metrics = [
    'hardest_boulder_ever',
    'hardest_boulder_confident',
    'hardest_boulder_recent',
    'hardest_route_ever',
    'hardest_route_recent',
    'hardest_route_confident',
    'pullups_maxcount_binned',
    'pushups_maxcount_binned'
]

violin_features = [
    'height_binned',
    'weight_binned',
    'wingspan_binned',
    'years_climbing',
    'indoor_outdoor',
    'sex',
    'pullups_maxcount_binned',
    'pushups_maxcount_binned'
]

corr_features = [
    'hardest_boulder_confident',
    'hardest_route_confident',
    'height',
    'weight',
    'wingspan',
    'pullups_maxcount',
    'pushups_maxcount',
    'weekly_count_campus',
    'weekly_hours_campus',
    'weekly_count_endurance',
    'weekly_count_strength',
    'session_hours_strength'
]

boulder = ["I don't boulder"]+[f'V{i}' for i in range(15)]
route = ["I don't climb routes"]+[str(i) for i in range(11, 34)]
widget_distincts = {
    'other_activities': [i for i in pd.unique(df.other_activities.explode()) if i],
    'hangboard_grips': [i for i in pd.unique(df.hangboard_grips.explode()) if i],
    'hangboard_style': [i for i in pd.unique(df.hangboard_style.explode()) if i],
    'endurance_style': [i for i in pd.unique(df.endurance_style.explode()) if i],
    'strength_style': [i for i in pd.unique(df.strength_style.explode()) if i],
    'height': [150.0, 201.1],
    'weight': [i for i in list(pd.unique(df.weight)) if i],
    'wingspan': [130.0, 208.0],
    'years_climbing': sorted_years,
    'hardest_boulder_confident': boulder,
    'hardest_boulder_recent': boulder,
    'hardest_boulder_ever': boulder,
    'hardest_route_confident': route,
    'hardest_route_recent': route,
    'hardest_route_ever': route,
    'strength_metrics': [key2label[i] for i in strength_metrics],
    'cid': list(df.cid.values),
    'indoor_outdoor': list(pd.unique(df.indoor_outdoor)),
    'sex': ['M', 'F']
}

def get_bins(series, n_bins, unit):
    min = series.min()
    max = series.max()
    bin_size = (max - min)/n_bins
    levels = [min+n*bin_size for n in range(n_bins+1)]

    bins = []

    for i in range(len(levels)):
        if i < len(levels)-1:
            start = int(levels[i])
            end = int(levels[i+1])

            bins.append(f"{start}-{end} {unit}")

    return bins

bin_me = [
    ('height', 'cm'),
    ('weight', 'kg'),
    ('wingspan', 'cm'),
    ('pullups_maxcount', 'reps'),
    ('pushups_maxcount', 'reps')
]
for feature, unit in bin_me:
    widget_distincts[feature+'_binned'] = get_bins(df[feature], 8, unit)


continuous_features = [
    'height',
    'weight',
    'wingspan',
    'pullups_maxcount',
    'pushups_maxcount',
    'weekly_count_campus',
    'weekly_hours_campus',
    'weekly_count_endurance',
    'weekly_count_strength',
    'session_hours_strength'
]

boulder_map = {f'V{i}': i for i in range(15)}
boulder_map["I don't boulder"] = None

route_map = {str(i): i for i in range(11,34)}
route_map["I don't climb routes"] = None

conversion = {
    'hardest_boulder_ever': boulder_map,
    'hardest_boulder_recent': boulder_map,
    'hardest_boulder_confident': boulder_map,
    'hardest_route_ever': route_map,
    'hardest_route_recent': route_map,
    'hardest_route_confident': route_map,
}

config = {
    'widget_options': widget_distincts,
    'conversion': conversion,
    'multi_features': multi_features,
    'continuous_features': continuous_features,
    'violin_features': violin_features,
    'correx_features': corr_features
}

In [14]:
with open('../data/config.json', 'w') as f:
    json.dump(config, f)

In [12]:
df.sort_values('wingspan')

,timestamp,sex,height,weight,wingspan,years_climbing,indoor_outdoor,hardest_boulder_ever,hardest_boulder_recent,hardest_boulder_confident,...,strength_style,other_activities,pullups_maxcount,pushups_maxcount,cid,height_binned,weight_binned,wingspan_binned,pullups_maxcount_binned,pushups_maxcount_binned
558,25/07/2020 01:21:07,M,110.00,60.0,110.0,1-2 years,Indoor Climbing only,V6,V6,V5,...,[Antagonists],None,30.0,NaN,sturdy finger,110-121 cm,53-60 kg,110-122 cm,30-40 reps,NaN
119,30/01/2017 15:08:42,M,175.00,50.0,110.0,10-11 years,Indoor and outdoor climbing,V7,V7,V4,...,"[Legs, Core, Upper body pushing]","[yoga, running, biking, hiking]",10.0,20.0,pinch-hating chalk eater,166-178 cm,45-53 kg,110-122 cm,10-20 reps,11-23 reps
311,19/02/2017 08:54:28,F,156.00,42.0,130.0,1-2 years,Indoor and outdoor climbing,V0,V0,V0,...,[Core],None,NaN,NaN,frenzied gym bro,155-166 cm,38-45 kg,122-134 cm,NaN,NaN
477,15/10/2019 10:57:07,F,157.00,59.0,144.0,1-2 years,Indoor Climbing only,V4,V4,V2,...,None,None,NaN,NaN,chalk-snorting dirtbag,155-166 cm,53-60 kg,134-146 cm,NaN,NaN
355,20/04/2017 13:07:22,F,158.00,54.0,148.0,2-3 years,Indoor and outdoor climbing,I don't boulder,I don't boulder,I don't boulder,...,None,None,NaN,NaN,free jug lord,155-166 cm,53-60 kg,146-159 cm,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
503,16/10/2019 02:57:31,M,172.72,64.5,NaN,3-4 years,Indoor and outdoor climbing,V7,V7,V4,...,None,None,NaN,NaN,jug-loving slab monarch,166-178 cm,60-68 kg,NaN,NaN,NaN
572,17/11/2020 10:11:58,M,180.00,72.0,NaN,6-7 years,Indoor and outdoor climbing,V11,V10,V8,...,None,None,23.0,39.0,bomber jug lord,178-189 cm,68-75 kg,NaN,20-30 reps,35-47 reps
606,19/02/2023 03:31:04,M,NaN,NaN,NaN,2-3 years,Indoor Climbing only,V4,V4,V3,...,[Upper body pushing],None,NaN,NaN,static afficionado,NaN,NaN,NaN,NaN,NaN
612,26/07/2023 22:41:51,F,168.00,62.0,NaN,<1 year,Indoor Climbing only,V2,V2,V1,...,"[Legs, Core, Upper body pulling, Upper body pu...",None,NaN,NaN,kid pebble,166-178 cm,60-68 kg,NaN,NaN,NaN


In [8]:
df.height.max()

201.1